# In this notebook, we're going to cover some of the most fundamental concepts of tensor using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (a way to speed up your regular Python function)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself


# Introduction to Tensors

In [19]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.18.0


In [20]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [21]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [22]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [23]:
# Check the dimension of our vector
vector.ndim

1

In [24]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [25]:
matrix.ndim

2

In [26]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [27]:
# What's the number dimensions of another_matrix
another_matrix.ndim

2

In [28]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [29]:
tensor.ndim

3

What we've created so far:
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional is a vector)


In [30]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor


(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [31]:
#Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [54]:
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [55]:
unchangeable_tensor[0] = 7

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [56]:
unchangeable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

## Creating random tensor
Random tensors are tensors of some arbitrary size which contain random numbers

In [57]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

# Are they equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [58]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [59]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [60]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

⚒ **Exercise:** Read through Tensorflow documentation on random see generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensor and shuffle them

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjuction to determine the random sequence."


In [61]:
exercise_tensor_1 = tf.constant(5)
tf.random.shuffle(exercise_tensor_1)

<tf.Tensor: shape=(), dtype=int32, numpy=5>

In [62]:
exercise_tensor_2 = tf.constant([5])
tf.random.shuffle(exercise_tensor_2)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([5], dtype=int32)>

In [63]:
exercise_tensor_3 = tf.constant([5, 7])
tf.random.shuffle(exercise_tensor_3)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 7], dtype=int32)>

In [64]:
exercise_tensor_4 = tf.constant([[5, 7, 4, 2],
                                 [1, 2, 3, 4]])
tf.random.shuffle(exercise_tensor_4)

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[5, 7, 4, 2],
       [1, 2, 3, 4]], dtype=int32)>

In [65]:
exercise_tensor_5 = tf.constant([[[5, 7, 4, 2],
                                 [1, 2, 3, 4]],
                                 [[1, 1, 4, 4],
                                 [7, 2, 8, 4]]])
tf.random.shuffle(exercise_tensor_5)

<tf.Tensor: shape=(2, 2, 4), dtype=int32, numpy=
array([[[1, 1, 4, 4],
        [7, 2, 8, 4]],

       [[5, 7, 4, 2],
        [1, 2, 3, 4]]], dtype=int32)>

### Other ways to make tensors

In [66]:
# Create a tendor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [67]:
# Create a tensor of all zeroes
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and Tensorflow is that tensors can be run on a GPU (much faster for numerical computing)


In [68]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [69]:
A = tf.constant(numpy_A , shape=(2,3,4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [70]:
A.ndim, B.ndim

(3, 1)

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size


In [71]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [72]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [73]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor))
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [74]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [75]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [76]:
some_list[:1]

[1]

In [77]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [78]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1]
rank_4_tensor[:1, :1, :1, :1]
rank_4_tensor[:1, :1, :, :1]
rank_4_tensor[:1, :, :1, :1]
rank_4_tensor[:, :1, :1, :1]

<tf.Tensor: shape=(2, 1, 1, 1), dtype=float32, numpy=
array([[[[0.]]],


       [[[0.]]]], dtype=float32)>

In [79]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])

rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [80]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [81]:
some_list, some_list[-1]

([1, 2, 3, 4], 4)

In [82]:
# Get the last item of each of row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [83]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor = rank_2_tensor[:, :, tf.newaxis]
rank_3_tensor


<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [84]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = -1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [85]:
tf.expand_dims(rank_2_tensor, axis=0) # expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

In [86]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### Manipulating tensors (tensor operations)

**Basic operations**

`+`, `-`, `*`, `/`


In [87]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [88]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [89]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [90]:
# Subraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [91]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [92]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common operations.

There are two rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimension




In [93]:
# Matrix multiplicaiton in tensorflow
tf.matmul(tensor, tensor)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [94]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [35]:
# Matrix multiplicaiton with Python operator "@"
tensor * tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[  1,   4,   9],
        [ 16,  25,  36]],

       [[ 49,  64,  81],
        [100, 121, 144]],

       [[169, 196, 225],
        [256, 289, 324]]], dtype=int32)>

In [95]:
tensor.shape

TensorShape([2, 2])

In [98]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
# create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [97]:
# Try to matrix multiply tensors of same shape
X @ Y
tf.matmul(X, Y)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

📖 **Resource:** Info and example of matrix multioplication: https://www.mathsisfun.com/algebra/matrix-multiplying.html




In [99]:
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [100]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [101]:
X.shape, tf.reshape(Y, shape = (2, 3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [102]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [103]:
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [104]:
# Try to matrix multiply X by reshaped Y
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [105]:
tf.reshape(X, shape=(2,3)).shape, Y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

In [106]:
# Can do the same with trnasponse
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [107]:
# Try matrix multiplicaiton with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:

* `tf.matmul()`
* `tf.tensordot()`
* `@`



In [108]:
X, Y, tf.transpose(X)

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>)

In [109]:
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [110]:
tf.tensordot(tf.transpose(X), Y, axes=0)

<tf.Tensor: shape=(2, 3, 3, 2), dtype=int32, numpy=
array([[[[ 7,  8],
         [ 9, 10],
         [11, 12]],

        [[21, 24],
         [27, 30],
         [33, 36]],

        [[35, 40],
         [45, 50],
         [55, 60]]],


       [[[14, 16],
         [18, 20],
         [22, 24]],

        [[28, 32],
         [36, 40],
         [44, 48]],

        [[42, 48],
         [54, 60],
         [66, 72]]]], dtype=int32)>

In [111]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [112]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [113]:
# Check the values of Y, reshape Y and transposed Y
print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, (2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))


Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [114]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Generally, when performing matrix multiplicaiton on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to get satify the matrix multiplicaiton rules.

### Changing the datatype of a tensor

In [115]:
tf.__version__

'2.18.0'

In [116]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [117]:
C = tf.constant([7, 10])
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>,
 tf.int32)

In [118]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype= tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [119]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E, E.dtype


(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [120]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.


In [121]:
# Get the absolute values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [122]:
#Get the absolute value
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [124]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([70, 60, 53, 62,  7, 59, 99, 52, 85, 90, 88, 99, 91, 13, 76, 75, 51,
       83, 62, 22, 97, 53, 81, 37, 70, 74, 33, 98, 89,  1, 63, 45, 73, 28,
       43, 75, 28, 34, 37, 86, 65, 85, 55, 76, 33, 45, 83, 76, 38, 21])>

In [125]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [126]:
# Find the minimum
tf.reduce_min(E), np.min(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=1>, 1)

In [127]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [128]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=60>

In [129]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=3019>

⚒ **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using Tensorflow methods.

In [130]:
# Find the variance
tf.reduce_variance(E) # won't work

AttributeError: module 'tensorflow' has no attribute 'reduce_variance'

In [131]:
# To find the varianc eof our tensor, we need access to tensorflow_probability
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=655>

In [132]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))


<tf.Tensor: shape=(), dtype=float32, numpy=25.604602813720703>

In [133]:
tf.math.reduce_std(E)

TypeError: Input must be either real or complex. Received integer type <dtype: 'int64'>.

In [134]:
# Find the variance of our E tensor
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=655.5956420898438>

### Find the positional maximum and minimum


In [135]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F= tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [136]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [137]:
np.argmax(F)

42

In [138]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [139]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [140]:
# Check for equality
assert F[tf.argmax(F)] == tf.reduce_max(F)

In [141]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [142]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463667869567871>

### Squeezing a tensor (removing all single dimensions)

In [143]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [144]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [145]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### One hot encoding

In [146]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth = 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [147]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value ="yo, I love deep learning",  off_value = "I also like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo, I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo, I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'yo, I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'yo, I love deep learning']],
      dtype=object)>

### Squaring, log ,square root

In [148]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [149]:
# square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [150]:
# Find the square root (will error, method requires non-int type)
tf.sqrt(H)

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sqrt}}; Op<name=Sqrt; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sqrt] name: 

In [151]:
# Find the square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [152]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays.

🔑 **Note** One of the main differences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run in a GPU or TPU (for faster numerical processing)


In [153]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7. , 10. ]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [154]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [155]:
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [156]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [157]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [158]:
import tensorflow as tf
tf.config.list_physical_devices()
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [159]:
!nvidia-smi

Sun Mar  2 12:11:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             32W /   70W |     120MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

> 🔑 **Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whneever possible.

### Exercises

In [160]:
# 1. Create a vector, scalar, matrix, and tensor with values of your choosing using tf.constant
vector = tf.constant([4.1, 4.5, 6.7])
scalar = tf.constant(3)
matrix = tf.constant([[.1, .6],
                             [.5, .09]])
tensor = tf.constant([[[1, 2],
                              [5, 1]],
                              [[1, 2],
                              [5, 1]]
                             ])

vector, scalar, matrix, tensor

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([4.1, 4.5, 6.7], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.1 , 0.6 ],
        [0.5 , 0.09]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
 array([[[1, 2],
         [5, 1]],
 
        [[1, 2],
         [5, 1]]], dtype=int32)>)

In [161]:
# 2. Find the shape, rank, and size of the tensors you created in 1
tf.shape(vector), vector.shape, tf.rank(vector), vector.ndim, tf.size(vector)


tf.shape(scalar), scalar.shape, tf.rank(scalar), scalar.ndim, tf.size(scalar)


tf.shape(matrix), matrix.shape, tf.rank(matrix), matrix.ndim, tf.size(matrix)

tf.shape(tensor), tensor.shape, tf.rank(tensor), tensor.ndim, tf.size(tensor)

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 2, 2], dtype=int32)>,
 TensorShape([2, 2, 2]),
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 3,
 <tf.Tensor: shape=(), dtype=int32, numpy=8>)

In [162]:
# 3. Create two tensors containing random variables between 0 and 1 with shape [5, 300]
tf.random.set_seed(42)
random1 = tf.random.uniform(minval=0, maxval =1, shape=(5, 300))
random2 = tf.random.uniform(minval=0, maxval=1, shape=(5, 300))

In [47]:
# 4. Multiply the two tensors you created in 3 using matrix multiplicaiton
tf.matmul(random1, tf.transpose(random2))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.333435, 73.40498 , 77.15961 , 73.98368 , 80.90053 ],
       [75.14637 , 68.80438 , 74.24303 , 71.84184 , 75.60205 ],
       [79.7594  , 75.64456 , 77.79758 , 74.748726, 80.55984 ],
       [75.085266, 69.06408 , 74.30776 , 72.27615 , 76.05667 ],
       [85.05689 , 74.26629 , 78.00687 , 74.886795, 83.13417 ]],
      dtype=float32)>

In [48]:
# 5. Multiply the two tensors you created in 3 using dot product
tf.tensordot(random1, tf.transpose(random2), axes=1)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[80.333435, 73.40498 , 77.15961 , 73.98368 , 80.90053 ],
       [75.14637 , 68.80438 , 74.24303 , 71.84184 , 75.60205 ],
       [79.7594  , 75.64456 , 77.79758 , 74.748726, 80.55984 ],
       [75.085266, 69.06408 , 74.30776 , 72.27615 , 76.05667 ],
       [85.05689 , 74.26629 , 78.00687 , 74.886795, 83.13417 ]],
      dtype=float32)>

In [49]:
# 6. Create a tensor with random values between 0 and 1 with shape [224, 224, 3]
tf.random.set_seed(42)
random_3 = tf.random.uniform(minval=0, maxval = 1, shape=(224, 224, 3))
random_3

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.6645621 , 0.44100678, 0.3528825 ],
        [0.46448255, 0.03366041, 0.68467236],
        [0.74011743, 0.8724445 , 0.22632635],
        ...,
        [0.42612267, 0.09686017, 0.16105258],
        [0.1487099 , 0.04513884, 0.9497483 ],
        [0.4393103 , 0.28527975, 0.96971095]],

       [[0.73308516, 0.5657046 , 0.33238935],
        [0.8838178 , 0.87544763, 0.56711245],
        [0.8879347 , 0.47661996, 0.42041814],
        ...,
        [0.7716515 , 0.9116473 , 0.3229897 ],
        [0.43050945, 0.83253574, 0.45549798],
        [0.29816985, 0.9639522 , 0.3316357 ]],

       [[0.41132426, 0.2179662 , 0.53570235],
        [0.5112119 , 0.6484759 , 0.8894886 ],
        [0.42459428, 0.20189774, 0.85781324],
        ...,
        [0.02888799, 0.3995477 , 0.11355484],
        [0.68524575, 0.04945195, 0.17778492],
        [0.97627187, 0.79811585, 0.9411576 ]],

       ...,

       [[0.9019445 , 0.27011132, 0.8090267 ],
        [0.32

In [50]:
# 7. Find the min and max values of the tensor you created in 6
tf.reduce_min(random_3), tf.reduce_max(random_3)

(<tf.Tensor: shape=(), dtype=float32, numpy=3.5762786865234375e-07>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.9999979734420776>)

In [51]:
# 8. Create a tensor with random values of shape [1, 224, 224, 3] then squeeze it to change the shape to [224, 224, 3]
random_4 = tf.random.uniform(minval = 0, maxval = 1, shape=(1, 224, 224, 3))
random_4
tf.squeeze(random_4)

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.68789124, 0.48447883, 0.9309944 ],
        [0.252187  , 0.73115396, 0.89256823],
        [0.94674826, 0.7493341 , 0.34925628],
        ...,
        [0.10315919, 0.9291539 , 0.99506986],
        [0.51019716, 0.9048054 , 0.22966433],
        [0.6319853 , 0.254349  , 0.6146672 ]],

       [[0.5757792 , 0.06120789, 0.41327834],
        [0.2947657 , 0.06683052, 0.8412062 ],
        [0.9196522 , 0.28208435, 0.88919353],
        ...,
        [0.8421594 , 0.92932725, 0.61236584],
        [0.9058645 , 0.48668993, 0.28548574],
        [0.30320632, 0.06468201, 0.79784656]],

       [[0.4406476 , 0.32621968, 0.03153896],
        [0.65943134, 0.43695068, 0.3716799 ],
        [0.7005588 , 0.81334114, 0.86623895],
        ...,
        [0.26015413, 0.7758442 , 0.32498944],
        [0.81527984, 0.906564  , 0.17323983],
        [0.65714204, 0.83752465, 0.18657303]],

       ...,

       [[0.4058814 , 0.8737236 , 0.9235314 ],
        [0.52

In [52]:
# 9. Create a tensor with shape [10] using your own choice of values, then find the index which has the maximum values
random_5 = tf.constant([1, 3, 88, 22, 12, 33, 55, 66, 10, 12])
tf.argmax(random_5)

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [53]:
# 10. One-hot encode the tensor you created in 9
tf.one_hot(random_5, depth=10)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>